<a href="https://colab.research.google.com/github/bs3537/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/_Keras_DepFFNN_AirBNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/bs3537/airbnb/master/listings.csv')

In [3]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,2217,Ian,Mitte,Brunnenstr. Süd,52.534537,13.402557,Entire home/apt,60,4,118,2018-10-28,3.76,4,141
1,2695,Prenzlauer Berg close to Mauerpark,2986,Michael,Pankow,Prenzlauer Berg Nordwest,52.548513,13.404553,Private room,17,2,6,2018-10-01,1.42,1,0
2,3176,Fabulous Flat in great Location,3718,Britta,Pankow,Prenzlauer Berg Südwest,52.534996,13.417579,Entire home/apt,90,62,143,2017-03-20,1.25,1,220
3,3309,BerlinSpot Schöneberg near KaDeWe,4108,Jana,Tempelhof - Schöneberg,Schöneberg-Nord,52.498855,13.349065,Private room,26,5,25,2018-08-16,0.39,1,297
4,7071,BrightRoom with sunny greenview!,17391,Bright,Pankow,Helmholtzplatz,52.543157,13.415091,Private room,42,2,197,2018-11-04,1.75,1,26


In [0]:
#Drop unecessary columns
cleaned_df = df.drop(columns=['id', 'name', 'host_id', 'host_name', 'calculated_host_listings_count', 'last_review', 'availability_365', 'minimum_nights', 'reviews_per_month', 'longitude', 'latitude'])

In [5]:
cleaned_df.head()

,neighbourhood_group,neighbourhood,room_type,price,number_of_reviews
0,Mitte,Brunnenstr. Süd,Entire home/apt,60,118
1,Pankow,Prenzlauer Berg Nordwest,Private room,17,6
2,Pankow,Prenzlauer Berg Südwest,Entire home/apt,90,143
3,Tempelhof - Schöneberg,Schöneberg-Nord,Private room,26,25
4,Pankow,Helmholtzplatz,Private room,42,197


In [6]:
cleaned_df.shape

(22552, 5)

In [7]:
cleaned_df.isnull().sum()

neighbourhood_group    0
neighbourhood          0
room_type              0
price                  0
number_of_reviews      0
dtype: int64

In [8]:
cleaned_df.dtypes

neighbourhood_group    object
neighbourhood          object
room_type              object
price                   int64
number_of_reviews       int64
dtype: object

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

In [10]:
train, val = train_test_split(cleaned_df, train_size=0.80, test_size=0.20, 
                              random_state=42)
train.shape, val.shape

((18041, 5), (4511, 5))

In [11]:
#Baseline for the train dataset
train['price']. describe()

count    18041.000000
mean        67.918852
std        228.548996
min          0.000000
25%         30.000000
50%         45.000000
75%         70.000000
max       9000.000000
Name: price, dtype: float64

In [12]:
guess = train['price'].mean()
print('At the baseline, the mean listing price is=', guess)

At the baseline, the mean listing price is= 67.91885150490549


In [0]:
#If we had just guessed the listing price, we would be off by how much?

In [14]:
from sklearn.metrics import mean_absolute_error

# Arrange y target vectors
target = 'price'
y_train = train[target]
y_val = val[target]

# Get mean baseline
print('Mean Baseline (using 0 features)')
guess = y_train.mean()

# Train Error
y_pred = [guess] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train mean absolute error: {mae:.2f} ')

# Test Error
y_pred = [guess] * len(y_val)
mae = mean_absolute_error(y_val, y_pred)
print(f'Validation mean absolute error: {mae:.2f} ')

Mean Baseline (using 0 features)
Train mean absolute error: 42.53 
Validation mean absolute error: 38.21 


In [0]:
#Fitting the ML model

In [0]:
# The status_group column is the target
target = 'price'

# Get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 200
categorical_features = cardinality[cardinality <= 200].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

In [0]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]

In [18]:
pip install category_encoders

In [0]:

import category_encoders as ce
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn import model_selection, preprocessing

In [0]:
encoder = OrdinalEncoder()

In [0]:
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.fit_transform(X_val)

In [0]:
X_train_encoded = np.array(X_train_encoded)

X_val_encoded = np.array(X_val_encoded)



In [0]:
#The dataset is clean, so no need for SimpleImputer

In [0]:
#MinMaxScaler

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train_encoded)

X_val_scaled = scaler.transform(X_val_encoded)

In [0]:
#Normalizer

from sklearn.preprocessing import Normalizer

norm = Normalizer()

X_train_norm = norm.fit_transform(X_train_scaled)

X_val_norm = scaler.transform(X_val_scaled)


In [0]:
X_train_norm = np.array(X_train_norm)

X_val_norm = np.array(X_val_norm)

In [27]:
X_train_norm.shape

(18041, 4)

In [28]:
X_val_norm.shape

(4511, 4)

In [0]:
#Keras NN Model

In [0]:
import tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [0]:
from tensorflow.keras.callbacks import EarlyStopping

In [0]:
model = Sequential()

#Stacking layers on model

#Keras Deep Feed Forward Neural Network

model.add(Dense(128, activation='relu', input_dim=4))
model.add(Dropout(0.2)) 

model.add(Dense(64, activation='relu')) #half the number of nodes in progressive hidden layers
model.add(Dropout(0.2)) 

model.add(Dense(32, activation='relu')) #half the number of nodes in progressive hidden layers
model.add(Dropout(0.2))

model.add(Dense(1, activation='linear'))   #use linear activation function in output layer for regression problems



In [0]:
es = EarlyStopping(monitor='val_mae', mode='min', verbose=1)

In [42]:
model.compile(loss='mean_absolute_error', optimizer= 'adam', metrics=['mae']) 

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 128)               640       
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_8 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                

In [43]:
model.fit(X_train_encoded, y_train, epochs=1000, verbose=2, validation_data=(X_val_encoded, y_val), callbacks=[es]) 

Epoch 1/1000
564/564 - 1s - loss: 42.3403 - mae: 42.3425 - val_loss: 34.8431 - val_mae: 34.8424
Epoch 2/1000
564/564 - 1s - loss: 37.6048 - mae: 37.6105 - val_loss: 29.2899 - val_mae: 29.2889
Epoch 3/1000
564/564 - 1s - loss: 34.0812 - mae: 34.0876 - val_loss: 28.8171 - val_mae: 28.8167
Epoch 4/1000
564/564 - 1s - loss: 33.5867 - mae: 33.5914 - val_loss: 27.8561 - val_mae: 27.8556
Epoch 5/1000
564/564 - 1s - loss: 33.3253 - mae: 33.3231 - val_loss: 27.8882 - val_mae: 27.8879
Epoch 00005: early stopping


##Model achieved similar Val. MAE as TPOTRegresser and RFClassifier_RFSearchCV

##No need to use scaling or normalization in regression problems where certain features are encoded as categorical variables. Doing so gives very bad Val. MAE.

###Results were improved on removing longitude and latitude. 

Final features used = Neighbourhood, Neighbourhood type, Number of reviews, Property Type